In [1]:
import pandas as pd
import numpy as np
import torch

import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/인공지능 프로젝트/project/mbti_1.csv',
                   engine='python', error_bad_lines=False)
print(len(data))
print(data.head(5))

8675
   type                                              posts
0  INFJ  'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1  ENTP  'I'm finding the lack of me in these posts ver...
2  INTP  'Good one  _____   https://www.youtube.com/wat...
3  INTJ  'Dear INTP,   I enjoyed our conversation the o...
4  ENTJ  'You're fired.|||That's another silly misconce...


In [5]:
idx2type = ['IE', 'NS', 'TF', 'JP']

def get_one_hot(row):
    t = row['type']
    
    I = int(t[0] == 'I')
    N = int(t[1] == 'N')
    T = int(t[2] == 'T')
    J = int(t[3] == 'J')

    return pd.Series({'IE': I, 'NS': N , 'TF': T, 'JP': J}) 

data = data.join(data.apply(lambda row: get_one_hot(row), axis=1))
print(data.head(5))

   type                                              posts  IE  NS  TF  JP
0  INFJ  'http://www.youtube.com/watch?v=qsXHcwe3krw|||...   1   1   0   1
1  ENTP  'I'm finding the lack of me in these posts ver...   0   1   1   0
2  INTP  'Good one  _____   https://www.youtube.com/wat...   1   1   1   0
3  INTJ  'Dear INTP,   I enjoyed our conversation the o...   1   1   1   1
4  ENTJ  'You're fired.|||That's another silly misconce...   0   1   1   1


In [6]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

tokenizer = None

class MBTI_Dataset(Dataset):
    def __init__(self, df):
        self.labels = torch.tensor(df[['IE', 'NS', 'TF', 'JP']].to_numpy())
        self.posts = [tokenizer(post, padding='max_length', max_length = 512,
                                truncation=True, return_tensors="pt") 
                        for post in df['posts']]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        post = self.posts[idx]
        label = self.labels[idx]
        return post, label

In [7]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=321)

## SequenceClassification (one classification a model)

In [8]:
target_type = 3

In [9]:
from transformers import AutoModelForSequenceClassification

model_name = 'distilbert-base-multilingual-cased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'pre_classifie

In [10]:
train_set = MBTI_Dataset(train_data.reset_index(drop=True))
train_loader = DataLoader(train_set, batch_size=2, shuffle=True, num_workers=2)
test_set = MBTI_Dataset(test_data.reset_index(drop=True))
test_loader = DataLoader(test_set, batch_size=2, shuffle=False, num_workers=2)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

num_epoch = 1

model.train()
for epoch in range(1, num_epoch + 1):
    stat_loss = 0

    for i, (posts, labels) in enumerate(train_loader, 1):
        optimizer.zero_grad()

        posts, labels = posts.to(device), labels[:, target_type].to(device)
        input_id, mask = posts['input_ids'].squeeze(1), posts['attention_mask']

        output = model(input_id, mask, labels=labels)
        loss = output.loss
                
        stat_loss += loss.item()
        if i % 500 == 0:
            print(f'[{epoch:-02}, {i:-4}] loss: {stat_loss / 500:.4f}')
            stat_loss = 0

        loss.backward()
        optimizer.step()

[01,  500] loss: 0.6838
[01, 1000] loss: 0.6393
[01, 1500] loss: 0.5917
[01, 2000] loss: 0.5793
[01, 2500] loss: 0.5675
[01, 3000] loss: 0.5495


In [12]:
acc = 0
model.eval()

with torch.no_grad():
    for i, (posts, labels) in enumerate(test_loader, 1):
        posts, labels = posts.to(device), labels[:, target_type].to(device)
        input_id, mask = posts['input_ids'].squeeze(1), posts['attention_mask']

        output = model(input_id, mask)
        acc += (output.logits.argmax(dim=1) == labels).sum()
        
    print(f'{idx2type[target_type]} test accuracy: {acc / (len(test_loader) * 2):.2%}')

JP test accuracy: 74.48%


## SequenceClassification

In [13]:
from transformers import AutoModelForSequenceClassification

model_name = 'distilbert-base-multilingual-cased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=4, problem_type="multi_label_classification"
        ).to(device)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'pre_classifie

In [14]:
train_set = MBTI_Dataset(train_data.reset_index(drop=True))
train_loader = DataLoader(train_set, batch_size=2, shuffle=True, num_workers=2)
test_set = MBTI_Dataset(test_data.reset_index(drop=True))
test_loader = DataLoader(test_set, batch_size=2, shuffle=False, num_workers=2)

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

num_epoch = 1

model.train()
for epoch in range(1, num_epoch + 1):
    stat_loss = 0

    for i, (posts, labels) in enumerate(train_loader, 1):
        optimizer.zero_grad()

        posts, labels = posts.to(device), labels.to(device, torch.float)
        input_id, mask = posts['input_ids'].squeeze(1), posts['attention_mask']

        output = model(input_id, mask, labels=labels)
        loss = output.loss
                
        stat_loss += loss.item()
        if i % 500 == 0:
            print(f'[{epoch:-02}, {i:-4}] loss: {stat_loss / 500:.4f}')
            stat_loss = 0

        loss.backward()
        optimizer.step()

[01,  500] loss: 0.5858
[01, 1000] loss: 0.5326
[01, 1500] loss: 0.4971
[01, 2000] loss: 0.4727
[01, 2500] loss: 0.4488
[01, 3000] loss: 0.4405


In [17]:
acc = 0
loss = 0
model.eval()

with torch.no_grad():
    for i, (posts, labels) in enumerate(test_loader, 1):
        posts, labels = posts.to(device), labels.to(device, torch.float)
        input_id, mask = posts['input_ids'].squeeze(1), posts['attention_mask']

        output = model(input_id, mask, labels=labels)
        acc += ((torch.sigmoid(output.logits) > 0.5) == labels).sum()
        loss += output.loss
        
    print(f'test accuracy: {acc / len(test_loader) / 2 / 4:.2%}')
    print(f'loss: {loss / len(test_loader)}')

test accuracy: 78.97%
loss: 0.45753130316734314


## Custom

In [18]:
from transformers import AutoModel

model_name = 'distilbert-base-multilingual-cased'

class CustomClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = AutoModel.from_pretrained(model_name)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(768, 768),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
        )
        self.layer3 = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attention_mask):
        y = self.layer1(input_ids, attention_mask).last_hidden_state[:, 0]
        y = self.layer2(y)
        y = self.layer3(y)
        return y

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = CustomClassifier().to(device)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
train_set = MBTI_Dataset(train_data.reset_index(drop=True))
train_loader = DataLoader(train_set, batch_size=2, shuffle=True, num_workers=2)
test_set = MBTI_Dataset(test_data.reset_index(drop=True))
test_loader = DataLoader(test_set, batch_size=2, shuffle=False, num_workers=2)

In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()

num_epoch = 1

model.train()
for epoch in range(1, num_epoch + 1):
    stat_loss = 0

    for i, (posts, labels) in enumerate(train_loader, 1):
        optimizer.zero_grad()

        posts, labels = posts.to(device), labels.to(device, torch.float)
        input_id, mask = posts['input_ids'].squeeze(1), posts['attention_mask']

        output = model(input_id, mask)
        loss = criterion(output, labels)
                
        stat_loss += loss.item()
        if i % 500 == 0:
            print(f'[{epoch:-02}, {i:-4}] loss: {stat_loss / 500:.4f}')
            stat_loss = 0

        loss.backward()
        optimizer.step()

[01,  500] loss: 0.5985
[01, 1000] loss: 0.5698
[01, 1500] loss: 0.4931
[01, 2000] loss: 0.4840
[01, 2500] loss: 0.4698
[01, 3000] loss: 0.4544


In [21]:
acc = 0
loss = 0
model.eval()

with torch.no_grad():
    for i, (posts, labels) in enumerate(test_loader, 1):
        posts, labels = posts.to(device), labels.to(device, torch.float)
        input_id, mask = posts['input_ids'].squeeze(1), posts['attention_mask']

        output = model(input_id, mask)
        acc += ((torch.sigmoid(output) > 0.5) == labels).sum()
        loss += criterion(output, labels)
        
    print(f'test accuracy: {acc / len(test_loader) / 2 / 4:.2%}')
    print(f'loss: {loss / len(test_loader)}')

test accuracy: 79.67%
loss: 0.4502811133861542
